# SageMaker/DeepAR demo on household electricity consumption dataset

This notebook complements the following two notebooks:
* [DeepAR introduction notebook](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_amazon_algorithms/deepar_synthetic/deepar_synthetic.ipynb). 
* [Individual household electric power consumption dataset](https://github.com/amirrezaeian/Individual-household-electric-power-consumption-Data-Set-/blob/master/data_e_power.ipynb).

The household electric power consumption dataset is available at:<br>
http://archive.ics.uci.edu/ml/datasets/Individual+household+electric+power+consumption

In summary, the dataset consists of the following attributes:
* `date`: date (dd/mm/yyyy)
* `time`: time (hh:mm:ss)
* `global_active_power`: household global minute-averaged active power (in Kilowatt) 
* `global_reactive_power`: household global minute-averaged reactive power (in Kilowatt)
* `voltage`: minute-averaged voltage (in Volt)
* `global_intensity`: household global minute-averaged current intensity (in Ampere) 
* `sub_metering_1`: energy sub-metering No.1 (in Watt-per-hour of active energy). It corresponds to the kitchen, containing mainly a dishwasher, an oven and a microwave (hot plates are not electric but gas powered). 
* `sub_metering_2`: energy sub-metering No.2 (in Watt-per-hour of active energy). It corresponds to the laundry room, containing a washing-machine, a tumble-drier, a refrigerator and a light. 
* `sub_metering_3`: energy sub-metering No.3 (in Watt-per-hour of active energy). It corresponds to an electric water-heater and an air-conditioner. 

In particular, we will see how to:
* Prepare the dataset
* Use the SageMaker Python SDK to train a DeepAR model and deploy it
* Make requests to the deployed model to obtain forecasts interactively
* Illustrate advanced features of DeepAR: missing values, additional time features, non-regular frequencies and category information

For more information about DeepAR, see the [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar.html) 

In [ ]:
%matplotlib inline

import sys
import os
import json
import zipfile
import random
import datetime
from urllib.request import urlretrieve
from dateutil.parser import parse
from random import shuffle

import boto3
import s3fs
import sagemaker
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import randint
import seaborn as sns # used for plot interactive graph. 

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import IntSlider, FloatSlider, Checkbox

from sklearn.model_selection import train_test_split # to split the data into two parts
from sklearn.cross_validation import KFold # use for cross validation
from sklearn.preprocessing import StandardScaler # for normalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline # pipeline making
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics # for the check the error and accuracy of the model
from sklearn.metrics import mean_squared_error,r2_score

## for Deep-learing:
import keras
from keras.layers import Dense
from keras.models import Sequential
# from keras.utils import to_categorical
from keras.optimizers import SGD 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
import itertools
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dropout


In [ ]:
# set random seeds for reproducibility
np.random.seed(42)
random.seed(42)

In [ ]:
sagemaker_session = sagemaker.Session()

Before starting, we can override the default values for the following: 
- The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.

In [ ]:
s3_bucket = '<put your S3 bucket name>'  # replace with an existing bucket if needed

s3_prefix = 'deepar-household-electricity-notebook'    # prefix used for all data stored within the bucket

role = sagemaker.get_execution_role()             # IAM role to use by SageMaker

In [ ]:
region = sagemaker_session.boto_region_name

s3_data_path = "s3://{}/{}/data".format(s3_bucket, s3_prefix)
s3_output_path = "s3://{}/{}/output".format(s3_bucket, s3_prefix)

Next, we configure the container image to be used for the region that we are running in.

In [ ]:
image_name = sagemaker.amazon.amazon_estimator.get_image_uri(region, "forecasting-deepar", "latest")

### Importing household electricity consumption dataset

After downloading the original dataset from the UCI ML repository, we load and parse the dataset. In addition, we modify dataset into a time-series formation supported by Pandas, so that we can utilize many features to handle time-series dataset (e.g. datetime, resampling, aggregation, basic statistics, etc.).

* https://www.allaboutcircuits.com/textbook/alternating-current/chpt-11/true-reactive-and-apparent-power/
* https://circuitglobe.com/what-is-power-triangle.html
* https://en.wikipedia.org/wiki/AC_power

<img src="./power-triangle-compressor.jpg" width=400>

### White noise
In discrete time, white noise is a discrete signal whose samples are regarded as a sequence of serially uncorrelated random variables with zero mean and finite variance. Depending on the context, one may also require that the samples be independent and having identical probability distribution (a.k.a. _i.i.d_). In particular, if each sample has a normal distribution with zero mean, the signal is said to be **Gaussian white noise**. 

Some properties of white noise:
* White noise is the simplest example of a **stationary process**. <br>
* if the lag is 0, auto-covariance will be a variance of probability distribution. Otherwise, auto-covariance will be 0. That is:
<br><br>
\begin{equation}
   \gamma_l = 
    \begin{cases}
        Var[e_t] & \text{for $l = 0$} \\
        0 & \text{for $l \neq 0$} 
    \end{cases}
\end{equation}
<br>
* if the lag is 0, auto-correlation will be 1. Otherwise, auto-correlation will be 0. That is:
<br><br>

\begin{equation}
   \rho_l = 
    \begin{cases}
        1 & \text{for $l = 0$} \\
        0 & \text{for $l \neq 0$} 
    \end{cases}
\end{equation}
<br>
* Gaussian white noise can be expressed by:
<br><br>
\begin{equation}
e_t \sim \text{ $i.i.d$ } N(\mu,\sigma^2) \text{ for all $t$}
\end{equation}

**Prewhitening:**
A technique to process a time series data to make it behave statistically like white noise. (The 'pre' means that whitening precedes some other analytical approaches enabling to work better if the additive noise is white).

https://datascienceschool.net/view-notebook/6b963e771dc54f8c8cb23437274a86d6/ <br>
http://hosting.astro.cornell.edu/~cordes/A6523/Prewhitening.pdf

In [ ]:
FILE_NAME = './household_power_consumption.csv'

* Note that dataset includes `nan` and `?` as a `string`. They need to be converted to numpy `nan` in importing stage and treated both of them the same.
* Two columns `Date` and `Time` can be merged into one column `Date_Time`.
* The index of dataset need to be reset (with `Date_Time`).

* Data can be downloaded from: http://archive.ics.uci.edu/ml/machine-learning-databases/00235/
* Just open the zip file and grab the file 'household_power_consumption.txt' put it in the directory that you would like to run the code.
* `infer_datetime_format`: to allow speedups for homogeneously formatted datetimes. `pd.read_csv` and `pd.to_datetime` learned a new `infer_datetime_format` keyword which greatly improves parsing perf in many cases. (http://pandas.pydata.org/pandas-docs/version/0.17.1/whatsnew.html#id55)
* `low_memory`: Please refer the following link: https://stackoverflow.com/questions/24251219/pandas-read-csv-low-memory-and-dtype-options)

In [ ]:
!wget http://archive.ics.uci.edu/ml/machine-learning-databases/00235/household_power_consumption.zip
!unzip household_power_consumption.zip
!mv household_power_consumption.txt household_power_consumption.csv

data = pd.read_csv(FILE_NAME, sep=",", parse_dates={'Date_Time': ['Date', 'Time']}, 
                  infer_datetime_format=True, na_values=['nan','?'], 
                  low_memory=False, index_col='Date_Time')
data.head()

We need to check the type of data for each column. If some of them has `object` type, they should be converted into the numerical format (e.g. `float64`, `int64`).
For example, we can use the following codes for the above tasks: <br>
```
data = data.convert_objects(convert_numeric=True)
data.info()
```

In [ ]:
data.info()

When we want to build an ML model, we need to understand dataset in the following perspectives:
* The meaning of data for each feature (column)
* Summarized information from the basic statistics
* Relaionship or Association between features
* Features having similar patterns
* Trend or Periodicity
* Outliers, Noisy data, Missing values
* Data type (categorical data, numerical data, etc.)
... 

In [ ]:
data.describe()

## Dealing with missing values 'nan' with a test statistic

In [ ]:
## finding all columns that have nan:

droping_list_all=[]
for j in range(0,7):
    if not data.iloc[:, j].notnull().all():
        droping_list_all.append(j)        
        #print(df.iloc[:,j].unique())
# droping_list_all

In [ ]:
# filling nan with mean in any columns

for j in range(0,7):        
        data.iloc[:,j]=data.iloc[:,j].fillna(data.iloc[:,j].mean())

In [ ]:
# another sanity check to make sure that there are not more any nan
data.isnull().sum()
data.describe()

We can make a transformed dataset with different frequency by using `resample()`. <br>
**down-sampling:** <br>
* To transform the original dataset to a lower frequencey
* Summarize or aggregate the higher frequency data points (i.e. original dataset)
* Example: 1 minute-based timestamps → 5 minute-based timestamps 

**up-sampling:** <br>
* To transform the original dataset to a higher frequencey
* the lower frequency data points (i.e. original dataset)
* Example: 1 minute-based timestamps → 0.5 minute-based (i.e. 30 second-based) timestamps 

https://machinelearningmastery.com/resample-interpolate-time-series-data-python/ <br>
**Resampling** <br>
Resampling involves changing the frequency of your time series observations.

Two types of resampling are:

1. Upsampling: Where you increase the frequency of the samples, such as from minutes to seconds. For upsampling, `ffill()` (i.e. forward filling) or `bfill()` (i.e. backward filling) can be required to fill the newly created data points that was not available. 
1. Downsampling: Where you decrease the frequency of the samples, such as from days to months. For downsampling, some kind of aggregation operation can be needed. (e.g. `mean()`, `first()`, etc.)

In both cases, data must be invented.

In the case of upsampling, care may be needed in determining how the fine-grained observations are calculated using interpolation. In the case of downsampling, care may be needed in selecting the summary statistics used to calculate the new aggregated values.

There are perhaps two main reasons why you may be interested in resampling your time series data:

1. Problem Framing: Resampling may be required if your data is available at the same frequency that you want to make predictions.
1. Feature Engineering: Resampling can also be used to provide additional structure or insight into the learning problem for supervised learning models.
There is a lot of overlap between these two cases.

For example, you may have daily data and want to predict a monthly problem. You could use the daily data directly or you could downsample it to monthly data and develop your model.

A feature engineering perspective may use observations and summaries of observations from both time scales and more in developing a model.


In [ ]:
data['Global_active_power'].resample('M').sum()

## Data Visualization

Below I resample over day, and show the sum and mean of Global_active_power. It is seen that mean and sum of resampled data set, have similar structure.

In [ ]:
data.Global_active_power.resample('D').sum().plot(title='Global_active_power resampled over day for sum') 
#df.Global_active_power.resample('D').mean().plot(title='Global_active_power resampled over day', color='red') 
plt.tight_layout()
plt.show()   

data.Global_active_power.resample('D').mean().plot(title='Global_active_power resampled over day for mean', color='red') 
plt.tight_layout()
plt.show()

Below I show mean and standard deviation (std) of 'Global_intensity' resampled over day.

In [ ]:
r = data.Global_intensity.resample('D').agg(['mean', 'std'])
r.plot(subplots = True, title='Global_intensity resampled over day')
plt.show()

Below I show mean and standard deviation (std) of 'Global_reactive_power' resampled over day.

In [ ]:
r2 = data.Global_reactive_power.resample('D').agg(['mean', 'std'])
r2.plot(subplots = True, title='Global_reactive_power resampled over day', color='red')
plt.show()

Below I show sum of 'global_active_power' resampled over day.

In [ ]:
# Sum of 'Global_active_power' resampled over month
data['Global_active_power'].resample('M').mean().plot(kind='bar')
plt.xticks(rotation=60)
plt.ylabel('Global_active_power')
plt.title('Global_active_power per month (averaged over month)')
plt.show()

Below I show mean of 'global_active_power' resampled over day.

In [ ]:
data['Global_active_power'].resample('Q').mean().plot(kind='bar')
plt.xticks(rotation=60)
plt.ylabel('Global_active_power')
plt.title('Global_active_power per quarter (averaged over quarter)')
plt.show()

It is very important to note from above two plots that resampling over larger time inteval, will diminish the periodicity of system as we expect. This is important for machine learning feature engineering.

mean of 'Voltage' resampled over month

In [ ]:
data['Voltage'].resample('M').mean().plot(kind='bar', color='red')
plt.xticks(rotation=60)
plt.ylabel('Voltage')
plt.title('Voltage per quarter (summed over quarter)')
plt.show()

In [ ]:
data['Sub_metering_1'].resample('M').mean().plot(kind='bar', color='brown')
plt.xticks(rotation=60)
plt.ylabel('Sub_metering_1')
plt.title('Sub_metering_1 per quarter (summed over quarter)')
plt.show()


It is seen from the above plots that the mean of 'Volage' over month is pretty much constant compared to other features. This is important again in feature selection.

### Comparison of the mean of different features resampled over day

In [ ]:
# Below I compare the mean of different featuresresampled over day. 
# specify columns to plot
cols = [0, 1, 2, 3, 5, 6]
i = 1
values = data.resample('D').mean().values
values

In [ ]:
# plot each column
groups = list(data.columns.values)
plt.figure(figsize=(15, 10))
for group in cols:
    plt.subplot(len(cols), 1, i)
    plt.plot(values[:, group])
    plt.title(data.columns[group], y=0.75, loc='right')
    i += 1
plt.show()

In [ ]:
## resampling over week and computing mean
data.Global_reactive_power.resample('W').mean().plot(color='y', legend=True)
data.Global_active_power.resample('W').mean().plot(color='r', legend=True)
data.Sub_metering_1.resample('W').mean().plot(color='b', legend=True)
data.Global_intensity.resample('W').mean().plot(color='g', legend=True)
plt.show()

In [ ]:
# Below I show hist plot of the mean of different feature resampled over month 
data.Global_active_power.resample('M').mean().plot(kind='hist', alpha=0.3, legend=True )
data.Global_reactive_power.resample('M').mean().plot(kind='hist', alpha=0.3, legend=True)
#df.Voltage.resample('M').sum().plot(kind='hist',color='g', legend=True)
data.Global_intensity.resample('M').mean().plot(kind='hist', alpha=0.3, legend=True)
data.Sub_metering_1.resample('M').mean().plot(kind='hist', alpha=0.3, legend=True)
plt.show()

In [ ]:
# Below I show hist plot of the mean of different feature resampled over month 
data.Global_active_power.resample('M').mean().plot(kind='hist', color='r', legend=True )
#from pyqt_fit import kde
plt.show()

### Comparison of the mean of different features resampled over day

In [ ]:
## The correlations between 'Global_intensity', 'Global_active_power'
# data_returns = data.pct_change()
sns.jointplot(x='Global_intensity', y='Global_active_power', data=data)  
plt.show()

In [ ]:
## The correlations between 'Voltage' and  'Global_active_power'
sns.jointplot(x='Voltage', y='Global_active_power', data=data)  
plt.show()

From above two plots it is seen that 'Global_intensity' and 'Global_active_power' correlated. But 'Voltage', 'Global_active_power' are less correlated. This is important observation for machine learning purpose.

### Comparison among features

In [ ]:
# Correlations among columns
plt.matshow(data.corr(method='spearman'),vmax=1,vmin=-1,cmap='RdBu')
plt.title('without resampling')
plt.colorbar()
plt.figure(figsize=(40,40))
plt.show()

### Comparison of mean of resampled features

In [ ]:
# Correlations of mean of features resampled over months
plt.matshow(data.resample('M').mean().corr(method='spearman'),vmax=1,vmin=-1,cmap='RdBu')
plt.title('resampled over month', size=15)
plt.colorbar()
plt.margins(0.02)
plt.matshow(data.resample('A').mean().corr(method='spearman'),vmax=1,vmin=-1,cmap='RdBu')
plt.title('resampled over year', size=15)
plt.colorbar()
plt.show()

It is seen from above that with resampling techniques one can change the correlations among features. This is important for feature engineering.

In [ ]:
display(data.corr())
pd.plotting.scatter_matrix(data, figsize=(12, 12))
plt.show()

Please check your scatter_matrix as below: <br>
<img src="./ScreenShot-2018-07-18-15-18-02.png" width="800"/>

#### StatsModels

`statsmodels` is a Python module that provides classes and functions for the estimation of many different statistical models, as well as for conducting statistical tests, and statistical data exploration. An extensive list of result statistics are available for each estimator. The results are tested against existing statistical packages to ensure that they are correct. The package is released under the open source Modified BSD (3-clause) license. The online documentation is hosted at [statsmodels.org](http://www.statsmodels.org/).

1. Statistics `stats`
    * statistical tests
    * kernel density estimation
    * generalized method of moments 
<br><br>
1. Linear regression
    * Linear model
    * Generalized Linear Model (GLM)
    * Robust Linear Model
    * Linear Mixed Effects Model
    * ANOVA (Analysis of Variance)
    * Discrete Dependent Variable 
<br><br>
1. Time-Series analysis
    * ARMA/ARIMA process
    * Vector ARMA process

In [ ]:
timeseries = data.Global_active_power.resample('D').mean()
# for i in range(num_timeseries):
#     timeseries.append(np.trim_zeros(data.iloc[:,i], trim='f'))

In [ ]:
timeseries.head()

In [ ]:
len(timeseries)

### Train and Test splits

Often times one is interested in evaluating the model or tuning its hyperparameters by looking at error metrics on a hold-out test set. Here we split the available data into train and test sets for evaluating the trained model. For standard machine learning tasks such as classification and regression, one typically obtains this split by randomly separating examples into train and test sets. However, in forecasting it is important to do this train/test split based on time rather than by time series.

In this example, we will reserve the last section of each of the time series for evalutation purpose and use only the first part as training data. 

In [ ]:
# we use minute frequency for the time series
freq = 'D'

# we predict for 60 days 
prediction_length = 60

# we also use 60 days as context length, 
# this is the number of state updates accomplished before making predictions
context_length = 60

We specify here the portion of the data that is used for training: the model sees data from 2006-12-16 to 2008-12-31 for training.

In [ ]:
timeseries.head()

In [ ]:
start_dataset = pd.Timestamp("2006-12-16", freq=freq)
end_training = pd.Timestamp("2010-07-11", freq=freq)

The DeepAR JSON input format represents each time series as a JSON object. In the simplest case each time series just consists of a start time stamp (``start``) and a list of values (``target``). For more complex cases, DeepAR also supports the fields ``dynamic_feat`` for time-series features and ``cat`` for categorical features, which we will use  later.

In [ ]:
training_data = [
    {
        "start": str(start_dataset),
        "target": timeseries[start_dataset:end_training - 1].tolist()  # We use -1, because pandas indexing includes the upper bound 
        # "target": ts[start_dataset:end_training - 1].tolist()  # We use -1, because pandas indexing includes the upper bound 
    }
    #for ts in timeseries
]
# print(len(training_data))

In [ ]:
len(timeseries[start_dataset:end_training -1])

In [ ]:
test_data = [
    {
        "start": str(start_dataset),
        "target": timeseries[end_training:end_training + 10 * prediction_length].tolist()  # We use -1, because pandas indexing includes the upper bound 
    }
    #for ts in timeseries
]
# print(len(test_data))

In [ ]:
len(timeseries[end_training:end_training + 10 * prediction_length])

Let's now write the dictionary to the `jsonlines` file format that DeepAR understands (it also supports gzipped jsonlines and parquet).

In [ ]:
def write_dicts_to_file(path, data):
    with open(path, 'wb') as fp:
        for d in data:
            fp.write(json.dumps(d).encode("utf-8"))
            fp.write("\n".encode('utf-8'))

In [ ]:
%%time
write_dicts_to_file("train.json", training_data)
write_dicts_to_file("test.json", test_data)

Now that we have the data files locally, let us copy them to S3 where DeepAR can access them. Depending on your connection, this may take a couple of minutes.

In [ ]:
s3 = boto3.resource('s3')
def copy_to_s3(local_file, s3_path, override=False):
    assert s3_path.startswith('s3://')
    split = s3_path.split('/')
    bucket = split[2]
    path = '/'.join(split[3:])
    buk = s3.Bucket(bucket)
    
    if len(list(buk.objects.filter(Prefix=path))) > 0:
        if not override:
            print('File s3://{}/{} already exists.\nSet override to upload anyway.\n'.format(s3_bucket, s3_path))
            return
        else:
            print('Overwriting existing file')
    with open(local_file, 'rb') as data:
        print('Uploading file to {}'.format(s3_path))
        buk.put_object(Key=path, Body=data)

In [ ]:
%%time
copy_to_s3("train.json", s3_data_path + "/train/train.json")
copy_to_s3("test.json", s3_data_path + "/test/test.json")

Let's have a look to what we just wrote to S3.

In [ ]:
s3filesystem = s3fs.S3FileSystem()
with s3filesystem.open(s3_data_path + "/train/train.json", 'rb') as fp:
    print(fp.readline().decode("utf-8")[:100] + "...")

We are all set with our dataset processing, we can now call DeepAR to train a model and generate predictions.

### Train a model

Here we define the estimator that will launch the training job.

In [ ]:
estimator = sagemaker.estimator.Estimator(
    sagemaker_session=sagemaker_session,
    image_name=image_name,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.m4.xlarge',
    base_job_name='deepar-home-electricity-demo',
    output_path=s3_output_path
)

Next we need to set the hyperparameters for the training job. For example frequency of the time series used, number of data points the model will look at in the past, number of predicted data points. The other hyperparameters concern the model to train (number of layers, number of cells per layer, likelihood function) and the training options (number of epochs, batch size, learning rate...). We use default parameters for every optional parameter in this case (you can always use [Sagemaker Automated Model Tuning](https://aws.amazon.com/blogs/aws/sagemaker-automatic-model-tuning/) to tune them).

In [ ]:
# hyperparameters = {
#     "time_freq": freq,
#     "epochs": "5",
#     "early_stopping_patience": "10",
#     "mini_batch_size": "20",
#     "learning_rate": "0.001",
#     "context_length": str(context_length),
#     "prediction_length": str(prediction_length)
# }
hyperparameters = {
    "time_freq": freq,
    "context_length": str(context_length),
    "prediction_length": str(prediction_length),
    "num_cells": "40",
    "num_layers": "3",
    "likelihood": "gaussian",
    "epochs": "20",
    "mini_batch_size": "32",
    "learning_rate": "0.001",
    "dropout_rate": "0.05",
    "early_stopping_patience": "10"
}

In [ ]:
estimator.set_hyperparameters(**hyperparameters)

We are ready to launch the training job. SageMaker will start an EC2 instance, download the data from S3, start training the model and save the trained model.

If you provide the `test` data channel as we do in this example, DeepAR will also calculate accuracy metrics for the trained model on this test. This is done by predicting the last `prediction_length` points of each time-series in the test set and comparing this to the actual value of the time-series. 

**Note:** the next cell may take a few minutes to complete, depending on data size, model complexity, training options.

In [ ]:
%%time
data_channels = {
    "train": "{}/train/".format(s3_data_path),
    "test": "{}/test/".format(s3_data_path)
}

estimator.fit(inputs=data_channels)

Since you pass a test set in this example, accuracy metrics for the forecast are computed and logged (see bottom of the log).
You can find the definition of these metrics from [our documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar.html). You can use these to optimize the parameters and tune your model or use SageMaker's [Automated Model Tuning service](https://aws.amazon.com/blogs/aws/sagemaker-automatic-model-tuning/) to tune the model for you.

### Create endpoint and predictor

Now that we have a trained model, we can use it to perform predictions by deploying it to an endpoint.

**Note: Remember to delete the endpoint after running this experiment. A cell at the very bottom of this notebook will do that: make sure you run it at the end.**

To query the endpoint and perform predictions, we can define the following utility class: this allows making requests using `pandas.Series` objects rather than raw JSON strings.

In [ ]:
class DeepARPredictor(sagemaker.predictor.RealTimePredictor):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, content_type=sagemaker.content_types.CONTENT_TYPE_JSON, **kwargs)
        
    def predict(self, ts, cat=None, dynamic_feat=None, 
                num_samples=100, return_samples=False, quantiles=["0.1", "0.5", "0.9"]):
        """Requests the prediction of for the time series listed in `ts`, each with the (optional)
        corresponding category listed in `cat`.
        
        ts -- `pandas.Series` object, the time series to predict
        cat -- integer, the group associated to the time series (default: None)
        num_samples -- integer, number of samples to compute at prediction time (default: 100)
        return_samples -- boolean indicating whether to include samples in the response (default: False)
        quantiles -- list of strings specifying the quantiles to compute (default: ["0.1", "0.5", "0.9"])
        
        Return value: list of `pandas.DataFrame` objects, each containing the predictions
        """
        prediction_time = ts.index[-1] + 1
        quantiles = [str(q) for q in quantiles]
        req = self.__encode_request(ts, cat, dynamic_feat, num_samples, return_samples, quantiles)
        res = super(DeepARPredictor, self).predict(req)
        return self.__decode_response(res, ts.index.freq, prediction_time, return_samples)
    
    def __encode_request(self, ts, cat, dynamic_feat, num_samples, return_samples, quantiles):
        instance = series_to_dict(ts, cat if cat is not None else None, dynamic_feat if dynamic_feat else None)

        configuration = {
            "num_samples": num_samples,
            "output_types": ["quantiles", "samples"] if return_samples else ["quantiles"],
            "quantiles": quantiles
        }
        
        http_request_data = {
            "instances": [instance],
            "configuration": configuration
        }
        
        return json.dumps(http_request_data).encode('utf-8')
    
    def __decode_response(self, response, freq, prediction_time, return_samples):
        # we only sent one time series so we only receive one in return
        # however, if possible one will pass multiple time series as predictions will then be faster
        predictions = json.loads(response.decode('utf-8'))['predictions'][0]
        prediction_length = len(next(iter(predictions['quantiles'].values())))
        prediction_index = pd.DatetimeIndex(start=prediction_time, freq=freq, periods=prediction_length)        
        if return_samples:
            dict_of_samples = {'sample_' + str(i): s for i, s in enumerate(predictions['samples'])}
        else:
            dict_of_samples = {}
        return pd.DataFrame(data={**predictions['quantiles'], **dict_of_samples}, index=prediction_index)

    def set_frequency(self, freq):
        self.freq = freq
        
def encode_target(ts):
    return [x if np.isfinite(x) else "NaN" for x in ts]        

def series_to_dict(ts, cat=None, dynamic_feat=None):
    """Given a pandas.Series object, returns a dictionary encoding the time series.

    ts -- a pands.Series object with the target time series
    cat -- an integer indicating the time series category

    Return value: a dictionary
    """
    obj = {"start": str(ts.index[0]), "target": encode_target(ts)}
    if cat is not None:
        obj["cat"] = cat
    if dynamic_feat is not None:
        obj["dynamic_feat"] = dynamic_feat        
    return obj

Now we can deploy the model and create and endpoint that can be queried using our custom DeepARPredictor class.

In [ ]:
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.m4.xlarge',
    predictor_cls=DeepARPredictor)

### Make predictions and plot results

Now we can use the `predictor` object to generate predictions.

In [ ]:
predictor.predict(timeseries[:-prediction_length], quantiles=[0.10, 0.5, 0.90])

Below we define a plotting function that queries the model and displays the forecast.

In [ ]:
def plot(
    predictor, 
    target_ts, 
    cat=None, 
    dynamic_feat=None, 
    forecast_date=end_training, 
    show_samples=False, 
    plot_history=7 * 12,
    confidence=80
):
    print("calling served model to generate predictions starting from {}".format(str(forecast_date)))
    assert(confidence > 50 and confidence < 100)
    low_quantile = 0.5 - confidence * 0.005
    up_quantile = confidence * 0.005 + 0.5
        
    # we first construct the argument to call our model
    args = {
        "ts": target_ts[:forecast_date],
        "return_samples": show_samples,
        "quantiles": [low_quantile, 0.5, up_quantile],
        "num_samples": 100
    }


    if dynamic_feat is not None:
        args["dynamic_feat"] = dynamic_feat
        fig = plt.figure(figsize=(20, 6))
        ax = plt.subplot(2, 1, 1)
    else:
        fig = plt.figure(figsize=(20, 3))
        ax = plt.subplot(1,1,1)
    
    if cat is not None:
        args["cat"] = cat
        ax.text(0.9, 0.9, 'cat = {}'.format(cat), transform=ax.transAxes)

    # call the end point to get the prediction
    prediction = predictor.predict(**args)

    # plot the samples
    if show_samples: 
        for key in prediction.keys():
            if "sample" in key:
                prediction[key].plot(color='lightskyblue', alpha=0.2, label='_nolegend_')
                
                
    # plot the target
    target_section = target_ts[forecast_date-plot_history:forecast_date+prediction_length]
    target_section.plot(color="black", label='target')
    
    # plot the confidence interval and the median predicted
    ax.fill_between(
        prediction[str(low_quantile)].index, 
        prediction[str(low_quantile)].values, 
        prediction[str(up_quantile)].values, 
        color="b", alpha=0.3, label='{}% confidence interval'.format(confidence)
    )
    prediction["0.5"].plot(color="b", label='P50')
    ax.legend(loc=2)    
    
    # fix the scale as the samples may change it
    ax.set_ylim(target_section.min() * 0.5, target_section.max() * 1.5)
    
    if dynamic_feat is not None:
        for i, f in enumerate(dynamic_feat, start=1):
            ax = plt.subplot(len(dynamic_feat) * 2, 1, len(dynamic_feat) + i, sharex=ax)
            feat_ts = pd.Series(
                index=pd.DatetimeIndex(start=target_ts.index[0], freq=target_ts.index.freq, periods=len(f)),
                data=f
            )
            feat_ts[forecast_date-plot_history:forecast_date+prediction_length].plot(ax=ax, color='g')

We can interact with the function previously defined, to look at the forecast of any customer at any point in (future) time. 

For each request, the predictions are obtained by calling our served model on the fly.

Here we forecast the consumption of an office after week-end (note the lower week-end consumption). 
You can select any time series and any forecast date, just click on `Run Interact` to generate the predictions from our served endpoint and see the plot.

In [ ]:
style = {'description_width': 'initial'}

In [ ]:
list_of_df = predictor.predict(timeseries[:-prediction_length])
actual_data = timeseries[-prediction_length:]

In [ ]:
len(actual_data)

In [ ]:
list_of_df

In [ ]:
timeseries.tail()

In [ ]:
plt.figure(figsize=(20,6))
timeseries[800:-len(list_of_df)].plot(color='#C3C8C4', linewidth=1.0)
p10 = list_of_df['0.1']
p90 = list_of_df['0.9']
plt.fill_between(p10.index, p10, p90, color='#C5F7AB', alpha=0.5, label='80% confidence interval')
actual_data.plot(color='#FCE08F', label='target')
list_of_df['0.5'].plot(marker='^', linewidth=3.0, label='prediction median')
plt.legend()
plt.show()

# Additional features

DeepAR has additional features such as handling the missing values as below:

* missing values: DeepAR can handle missing values in the time series during training as well as for inference.
* Additional time features: DeepAR provides a set default time series features such as hour of day etc. However, you can provide additional feature time series via the `dynamic_feat` field. 
* generalize frequencies: any integer multiple of the previously supported base frequencies (minutes `min`, hours `H`, days `D`, weeks `W`, month `M`) are now allowed; e.g., `15min`. We already demonstrated this above by using `2H` frequency.
* categories: If your time series belong to different groups (e.g. types of product, regions, etc), this information can be encoded as one or more categorical features using the `cat` field.


Based on the previous results, we will be able to impelement the advanced models to support the above features.

### Delete endpoints

In [ ]:
predictor.delete_endpoint()